In [ ]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

scaler = StandardScaler()


In [ ]:
# create quadratic dataset
def create_data_points(m) :
    x = 6 * np.random.rand(m, 1) - 3   # 100 random numbers between -3 and +3
    y = 0.5*x*x + x + 2                # parabola with three parameter
    y = y + np.random.randn(m, 1)      # add gaussian noise (randn versus rand)
    return x,y

x,y = create_data_points(100)
plt.plot(x,y,'.')
plt.show()

# The above would work with calls rand(m) instead of rand(m,1) but
# below (for fitting) we need x and y to be 2D-arrays (with one column)
x.shape

In [ ]:
# polynomial regression = linear regression on polynomial coefficients
poly_features = PolynomialFeatures(degree=2, include_bias=True)
x_poly = poly_features.fit_transform(x)
x_poly.shape                            # [x, x*x], with include_bias we'd have: [1.0, x, x*x]
lr = LinearRegression()                 # i.e. apply normal equation
lr.fit(x_poly, y)
print("lin reg found intercept:", lr.intercept_, ' coefficients: ', lr.coef_)

# more elegant two steps in a pipeline, exact same result:
pr = Pipeline([
    ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
    ("lr", LinearRegression())
])
pr.fit(x,y)
lr = pr.named_steps['lr']   # needed here only to access coefficients
print("pol reg found intercept:", lr.intercept_, ' coefficients: ', lr.coef_)

# use the pipeline-created model to predict
x_ = np.linspace(-3,3,20).reshape(20,1) # again we need a 2D array
y_ = pr.predict(x_)
plt.plot(x,y,'.')
plt.plot(x_,y_)
plt.show()

In [ ]:
# suche per batch, stochastic or minibatch and plot convergence
eta0 = 1.0       # I need to scale by 100, 1, 10 for batch, stoch, minibatch resp. 
                 # for acceptable stability. seems related to num points, but isn't really ?
miniSize = 10    # size of minibatch
epoch = len(x)


# create startpoint in lower left, plot 'S'
# two for each method Batch, Stochastic, Minibatch: the _p will be used for to collect step
# ts with constanet step, tsa is annealing, tm = minibatch is annealing only
tb = tbp = ts = tsp =tsa = tsap = tm = tmp = np.random.rand(3,1) - np.array([1,1,1]).reshape(3,1)
plt.text(tb[1], tb[2], 'S')

# we need offset = bias = x0 in our polyfeatures
x_poly = PolynomialFeatures(degree=2, include_bias=True).fit_transform(x)  # xi = [1.0, xi, xi*xi]

#  # havent quite understood yet: has a major effect: can use 10 times higher eta
#  # and convergence is no longer curved, but the resulting theta is a different one !!
#  x_poly = scaler.fit_transform(x_poly, y)

m = x_poly.shape[0]
print("m:", m)

# batch gradient descent (no sklearn implementation ?)
# NOTE: 5x more steps, each of them!!
for step in range(1,epoch) :
    gb = (2/m) * x_poly.T.dot(x_poly.dot(tb) - y)
    eta = eta0 /100
    tb = tb - eta*gb 
    tbp = np.hstack((tbp, tb))
    
# stochastic = one datapoint per step
for step in range(1,epoch) :
    ri = np.random.randint(len(x_poly))
    gs = (2/m) * x_poly[ri:ri+1].T.dot(x_poly[ri:ri+1].dot(ts) - y[ri:ri+1])
    eta =  eta0 
    ts = ts - eta*gs
    tsp = np.hstack((tsp, ts))

# stochastic (annealing) = one datapoint per step, eta decreasing
for step in range(1,epoch) :
    ri = np.random.randint(len(x_poly))
    gsa = (2/m) * x_poly[ri:ri+1].T.dot(x_poly[ri:ri+1].dot(tsa) - y[ri:ri+1])
    eta = ((0.99**step) * eta0 )
    tsa = tsa - eta*gsa
    tsap = np.hstack((tsap, tsa))

# minibatch = minibatch of datapoints per step
for step in range(1,epoch) :
    ri = np.random.randint(0,len(x_poly), miniSize)
    gm = (2/m) * x_poly[ri].T.dot(x_poly[ri].dot(tm) - y[ri])   
    eta = ((0.99**step) * eta0 )/ miniSize
    tm = tm - eta*gm
    tmp = np.hstack((tmp, tm))
    
plt.text(1.0, 0.5, '*')
plt.text(1.0, 0.0, 'batch', color='b')
plt.text(1.0, -0.2, 'stoch', color='r')
plt.text(1.0, -0.4, 'annealing')
plt.text(1.0, -0.6, 'minibatch', color='g')
plt.plot(tbp[1], tbp[2], 'b')
plt.plot(tsp[1], tsp[2], 'r')
plt.plot(tsap[1], tsap[2], 'k')
plt.plot(tmp[1], tmp[2], 'g')
        
plt.axis([-1,2,-1,2])
plt.grid(True)
plt.show()
# not perfect because we have only 100 datapoints
# especially offset is very unprecise
print ("batch:", tb)  
print ("target:", np.array([2, 1, 0.5]).reshape(3,1))


In [ ]:
# now we over-fit the data, by having polynomial with n>>2
def check_for_n(n) :
    
    model = Pipeline([
        ("pf", PolynomialFeatures(degree=n, include_bias=False)),
        ("lr", LinearRegression())
    ])

    # to validate we need training and validation set
    X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

    # we train first with just one point, then with increasingly more, 
    # for each training we calculate training- and validation error
    train_error, val_error = [], []  # these are python lists
    for m in range(1,len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_pred = model.predict(X_train[:m])
        y_val_pred = model.predict(X_val)
        train_error.append(mean_squared_error(y_train_pred, y_train[:m]))
        val_error.append(mean_squared_error(y_val_pred, y_val))
    
    plt.plot(np.sqrt(train_error), 'r-+')
    plt.plot(np.sqrt(val_error), 'b')
    plt.axis([0,len(x),0,3])
    plt.show()
    
x,y = create_data_points(100)
print("n=1: underfitted: error too high")
check_for_n(1)
print("n=2")
check_for_n(2)
print("n=10: overfitting: val-error > train-error")
check_for_n(5)

In [ ]:
# show the effect of regularisations: first a comparison of ridge (normal equation) 
# versus gradient descent 

def poly(x):
    return PolynomialFeatures(degree=8, include_bias=True).fit_transform(x)

def scaledPoly(x):
    return scaler.transform(poly(x))

# define the points where we want to plot predictions
x_ = np.linspace(-3,3,30).reshape(30,1) # 
scaler.fit(poly(x_))  

# create and plot datapoints on a parabola with noise
x,y = create_data_points(15)
plt.plot(x,y,'o')

# linear Regression (i.e. solving normal equation, not iterative)
ridge = Ridge(alpha=1e-3, solver="cholesky")
ridge.fit(scaledPoly(x),y)
y_ = ridge.predict(scaledPoly(x_))
plt.plot(x_,y_,'r-', linewidth=0.4, label="ridge alpha=1e-3")

ridge = Ridge(alpha=1, solver="cholesky")
ridge.fit(scaledPoly(x),y)
y_ = ridge.predict(scaledPoly(x_))
plt.plot(x_,y_,'r-', linewidth=0.8, label="ridge alpha=1")

# similar results withgradient descent SGDRegressor
# y must be a 1D list now (unstable without scaling!)
sgd_reg = SGDRegressor(penalty="l2", alpha=1e-3, tol=1E-5, max_iter=10000)
sgd_reg.fit(scaler.transform(scaledPoly(x)), y.ravel())
y_ = sgd_reg.predict(scaler.transform(scaledPoly(x_)))
plt.plot(x_,y_,'r.', linewidth=1, label="sgd ridge alpha=1e-3")


plt.legend()
plt.axis([-3,3,0,10])
plt.show()

In [ ]:
# now compare ridge to lasso to elastic net
# regularisation with l2-norm, l1-norm or a mixture of both
# I dont see major differences
x,y = create_data_points(10)

# ridge using normal equation
ridge = Ridge(alpha=1e-3, solver="cholesky")
ridge.fit(scaledPoly(x),y)
y_ = ridge.predict(scaledPoly(x_))
plt.plot(x_,y_,'b-', linewidth=0.5, label="ridge alpha=1e-3")

# lasso is by definition gradient descent ?
lasso = Lasso(alpha=1e-3, tol=1e-3, max_iter=10000)
lasso.fit(scaledPoly(x),y)
y_ = lasso.predict(scaledPoly(x_))
plt.plot(x_,y_,'g-', linewidth=0.5,  label="lasso")

# should be exact same result but isnt ?
lasso = SGDRegressor(penalty="l1", alpha=1e-3, tol=1e-3, max_iter=1000)
lasso.fit(scaledPoly(x),y.ravel())
y_ = lasso.predict(scaledPoly(x_))
plt.plot(x_,y_,'gx', linewidth=0.5,  label="sgd-lasso")

# elastic net
net = ElasticNet(alpha=1e-3, tol=1e-3, max_iter=10000, l1_ratio=0.5)
net.fit(scaledPoly(x),y)
y_ = net.predict(scaledPoly(x_))
plt.plot(x_,y_,'r-', linewidth=0.5, label="elastic net")

plt.plot(x,y,'bo')
plt.legend()
plt.axis([-3,3,0,10])
plt.show()

In [ ]:
# und jetzt noch den Iris Datensatz klassifizieren
# jeweils 4 Messergebnisse an 150 Blueten
iris = datasets.load_iris()
df = pandas.DataFrame(iris['data'], columns=iris['feature_names'])
print("features (1st rows):", df[:5])
print("target ist 1D numpy.array laenge 150")

# learn then classify one label one one feature
petalWidth = iris['data'][:,3].reshape(-1,1)  # need as 2D array, each feature a one-element row
isC = iris['target'] == 2
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(petalWidth, isC)  
predC = logreg.predict(petalWidth)
print("one feature one label, incorrect: ", (isC != predC).sum())

# now with all features, nothing really changes, exept better result
isC = iris['target'] == 2
x = iris['data']
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(x, isC)  
predC = logreg.predict(x)
print("all features one label, incorrect: ", (isC != predC).sum())



In [ ]:
# multiclass prediction

# In chapter 2 and 3 we just defined target as a one-hot encoded array: 
# this worke for Linear Regression, Random Foreset and stochastic gradient descent
# but it isn't accepted by LogisticRegression (probably because below method better)
# following gives error
# encoder = OneHotEncoder(categories='auto')
# y_as_onehot = encoder.fit_transform(iris['target'].reshape(-1,1))
# logreg.fit(x, y_as_onehot)

# Logistic Regression supports "multiclass" (see book): 
# decision based on score, (actuall softmax function of score)
# trained by minimizing cross entropy, the gradient of which is (p-y)*x, 

soft_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
x = iris["data"][:,(2,3)]  # two features = two columns
y = iris["target"]
soft_reg.fit(x, iris["target"])

# plot the result
colormapBG = ListedColormap(['#FFDDDD', '#DDFFDD', '#DDDDFF'])
colormapFG = ListedColormap(['#AA0000', '#00AA00', '#0000AA'])
plt.xlim(0,8)
plt.ylim(0,3)
xx, yy = np.meshgrid(np.arange(0,8,0.1), np.arange(0,3,0.1))
Z = soft_reg.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.pcolormesh(xx,yy,Z, cmap=colormapBG)
plt.scatter(x[:,0], x[:,1], c=y, cmap=colormapFG)
